# Code Testing Notebook
This notebook is for testing the finalized models and codes with minimal imports and no training involved.


## Importing Stage
### Libraries and Functions

In [1]:
import csv
import heapq
import pickle
import numpy as np
import pandas as pd

from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

2023-11-21 23:27:21.553581: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-21 23:27:21.553839: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-21 23:27:21.554403: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-21 23:27:21.578007: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Model and Wrapper Functions

In [2]:
model = load_model("./only_four_final_model.h5")
with open("./tokenizer.pickle", "rb") as file:
    tokenizer = pickle.load(file)


2023-11-21 23:27:24.967190: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-21 23:27:24.972936: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-21 23:27:24.973123: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
maxLen = 1163
def AutoCompleteSong(seedText: str, wordCount: int):
    for _ in range(wordCount):
        tokenList = tokenizer.texts_to_sequences([seedText])[0]
        tokenList = pad_sequences(
            [tokenList],
            maxlen = maxLen - 1,
            padding = "pre"
        )

        predictX = model.predict(
            tokenList,
            verbose = 0
        )
        pred = np.argmax(predictX, axis = 1)

        output = ""
        for word, index in tokenizer.word_index.items():
            if index == pred:
                output = word
                break

        seedText += " " + output
    return seedText

## Stage 1: Lyrics Generation

In [4]:
seedText = input("Enter initial lyrics")

In [5]:
songLen = int(input("Enter final length of song:"))

In [6]:
fullLyric = AutoCompleteSong(seedText, songLen)

2023-11-21 23:27:39.153932: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902


## Stage 2: Lyrics Matching

Embedding the completed lyrics using S-BERT model

In [7]:
transformer = SentenceTransformer('bert-base-nli-mean-tokens')
token = transformer.encode(fullLyric)

Loading stored embeddings and track lists

In [8]:
with open("./embeddings.pickle", "rb") as file:
	embeds = pickle.load(file)["embeddings"]

with open("./trackList.csv", "r") as file:
	trackList = list(csv.reader(file, delimiter = ","))[0]


Getting the 3 most similar songs

In [9]:
sim = cosine_similarity([token], embeds)[0]
songs = heapq.nlargest(3, zip(sim, trackList))
songs

[(0.875838, 'TRICITZ128F428516E'),
 (0.8739208, 'TRZXOWB12903CC3B24'),
 (0.87379754, 'TRCVVYK128F425AE08')]

The above cell's output has 3 pairs of following data information:
* Similarity Score
* Track ID

Extracting Track IDs from the result for further use

In [10]:
songSeq = []
for i in songs:
	songSeq.append(i[1])

songSeq

['TRICITZ128F428516E', 'TRZXOWB12903CC3B24', 'TRCVVYK128F425AE08']